In [1]:
import pandas as pd
import numpy as np

# New Section

https://github.com/lukasgarbas/nlp-text-emotion/blob/master/bert.ipynb

In [2]:
dataset = pd.read_csv('Bhaav-Dataset.csv')

# https://www.analyticsvidhya.com/blog/2021/12/fine-tune-bert-model-for-sentiment-analysis-in-google-colab/

https://swatimeena989.medium.com/bert-text-classification-using-keras-903671e0207d

In [3]:
import pandas as pd
import numpy as np

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses

In [5]:
dataset

,Sentences,Annotation
0,रमजान के पूरे तीस रोजों के बाद ईद आयी है,1
1,"कितना मनोहर, कितना सुहावना प्रभाव है",1
2,"वृक्षों पर अजीब हरियाली है, खेतों में कुछ अजीब...",1
3,"आज का सूर्य देखो, कितना प्यारा, कितना शीतल है,...",1
4,गाँव में कितनी हलचल है,1
...,...,...
20299,फिर यहाँ सर्दी हो जाती है,4
20300,दिन-भर मैं यह देखती रहती हूँ कि धूप का टुकड़ा क...,4
20301,"पार्क का कोई ऐसा कोना नहीं, जहाँ मैं घड़ी-आधा घ...",4
20302,लेकिन यह बेंच मुझे सबसे अच्छी लगती है,1


In [6]:
dataset['Annotation'].value_counts()

4    11697
2     3168
1     2463
3     1512
0     1464
Name: Annotation, dtype: int64

In [7]:
dataset['Emotion'] = np.where(
    dataset['Annotation'] == 0, 'Anger', np.where(
        dataset['Annotation'] == 1, 'Joy', np.where(
            dataset['Annotation'] == 2, 'Sad', np.where(
                dataset['Annotation'] == 3, 'Suspense', 'Neutral'))))
dataset

,Sentences,Annotation,Emotion
0,रमजान के पूरे तीस रोजों के बाद ईद आयी है,1,Joy
1,"कितना मनोहर, कितना सुहावना प्रभाव है",1,Joy
2,"वृक्षों पर अजीब हरियाली है, खेतों में कुछ अजीब...",1,Joy
3,"आज का सूर्य देखो, कितना प्यारा, कितना शीतल है,...",1,Joy
4,गाँव में कितनी हलचल है,1,Joy
...,...,...,...
20299,फिर यहाँ सर्दी हो जाती है,4,Neutral
20300,दिन-भर मैं यह देखती रहती हूँ कि धूप का टुकड़ा क...,4,Neutral
20301,"पार्क का कोई ऐसा कोना नहीं, जहाँ मैं घड़ी-आधा घ...",4,Neutral
20302,लेकिन यह बेंच मुझे सबसे अच्छी लगती है,1,Joy


In [8]:
'''
full_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(dataset['Sentences'].values, tf.string),
            tf.cast(dataset['Annotation'].values, tf.int32)
        )
    )
)
'''

"\nfull_dataset = (\n    tf.data.Dataset.from_tensor_slices(\n        (\n            tf.cast(dataset['Sentences'].values, tf.string),\n            tf.cast(dataset['Annotation'].values, tf.int32)\n        )\n    )\n)\n"

In [9]:
train_size = int(dataset.shape[0]*0.8)

In [10]:
dataset = dataset[['Sentences','Annotation']]

print(dataset.shape)
dataset

(20304, 2)


,Sentences,Annotation
0,रमजान के पूरे तीस रोजों के बाद ईद आयी है,1
1,"कितना मनोहर, कितना सुहावना प्रभाव है",1
2,"वृक्षों पर अजीब हरियाली है, खेतों में कुछ अजीब...",1
3,"आज का सूर्य देखो, कितना प्यारा, कितना शीतल है,...",1
4,गाँव में कितनी हलचल है,1
...,...,...
20299,फिर यहाँ सर्दी हो जाती है,4
20300,दिन-भर मैं यह देखती रहती हूँ कि धूप का टुकड़ा क...,4
20301,"पार्क का कोई ऐसा कोना नहीं, जहाँ मैं घड़ी-आधा घ...",4
20302,लेकिन यह बेंच मुझे सबसे अच्छी लगती है,1


In [11]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(dataset, test_size=0.20)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (16243, 2)
test shape:  (4061, 2)


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset['Sentences'], dataset['Annotation'], test_size=0.2)

In [13]:
class_names=list(set(y_train.values))
class_names
num_class_names=len(class_names)
num_class_names

5

In [14]:
!pip install -q transformers

In [15]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import TFBertModel, BertConfig

In [16]:

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [17]:
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=num_class_names)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
sent= 'how to train the model, lets look at how a trained model calculates its prediction.'

In [19]:
tokenized_sequence= bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =30,pad_to_max_length = True,
return_attention_mask = True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [20]:
tokenized_sequence

{'input_ids': [101, 14796, 10114, 17767, 10105, 13192, 117, 13595, 10107, 25157, 10160, 14796, 169, 33626, 13192, 87838, 26171, 10474, 78089, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [21]:
bert_tokenizer.decode(tokenized_sequence['input_ids'])

'[CLS] how to train the model, lets look at how a trained model calculates its prediction. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [22]:
sentences=dataset['Sentences']
labels=dataset['Annotation']
len(sentences),len(labels)

(20304, 20304)

In [23]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [24]:
len(input_ids),len(attention_masks),len(labels)

(20304, 20304, 20304)

In [25]:
import pickle
print('Preparing the pickle file.....')

pickle_inp_path='bert_inp.pkl'
pickle_mask_path='bert_mask.pkl'
pickle_label_path='bert_label.pkl'

pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

Preparing the pickle file.....
Pickle files saved as  bert_inp.pkl bert_mask.pkl bert_label.pkl


In [26]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (20304, 64) Attention mask shape (20304, 64) Input label shape (20304,)


In [27]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)


In [28]:
train_inp.shape

(16243, 64)

In [29]:
val_inp.shape

(4061, 64)

In [30]:
train_label.shape

(16243,)

In [31]:
val_label.shape

(4061,)

In [32]:
train_mask.shape

(16243, 64)

In [33]:
val_mask.shape

(4061, 64)

In [34]:
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Model
import keras.backend as K
from keras.models import load_model
log_dir='tensorboard_data/tb_bert'
model_save_path='bert_model.h5'
#callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True)]

In [35]:
print('\nBert Model',bert_model.summary())
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 177,857,285
Trainable params: 177,857,285
Non-trainable params: 0
_________________________________________________________________

Bert Model None


In [36]:
history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=2,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)
#history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=4,validation_data=([val_inp,val_mask],val_label))

Epoch 1/2
508/508 [==============================] - 11890s 23s/step - loss: 1.2427 - accuracy: 0.5750 - val_loss: 1.2298 - val_accuracy: 0.5705
Epoch 2/2
508/508 [==============================] - 11913s 23s/step - loss: 1.1737 - accuracy: 0.5770 - val_loss: 1.1849 - val_accuracy: 0.5765


NameError: ignored

In [6]:
!pip install -q transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import TFBertModel, BertConfig
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Model
import keras.backend as K
from keras.models import load_model
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses



model_save_path='bert_model.h5'

trained_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',num_labels=5)
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
#trained_model.load_weights(model_save_path)

preds = trained_model.predict([val_inp,val_mask],batch_size=32)
pred_labels = preds.argmax(axis=1)
f1 = f1_score(val_label,pred_labels)
print('F1 score',f1)
print('Classification Report')
print(classification_report(val_label,pred_labels,target_names=target_names))

print('Training and saving built model.....')   

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: ignored